In [ ]:
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:

!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric


In [ ]:
from torch.autograd import Variable
import torch_geometric.transforms
from torch_geometric.transforms import knn_graph
import torch_geometric.data
import torch 
from torch_geometric.data import Data
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
import torch
import pyarrow as pa
import pyarrow.parquet as pq
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from torch_geometric.nn import knn_graph
import os 

from torch.utils.data import *
from functools import partial
import timeit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import argparse
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=50, help='Number of epochs to train.')
parser.add_argument('--batch_size', type=int, default=50, help='Initial learning rate.') #100

parser.add_argument('--lr', type=float, default=0.001, help='Initial learning rate.') #0.001
parser.add_argument('--dropout', type=float, default=0.3, help='Dropout rate (1 - keep probability).')
args = parser.parse_args([])
torch.backends.cudnn.benchmark = True

In [ ]:
from torch.utils.data import *
class ParquetDataset(Dataset):
    def __init__(self, filename):
        self.parquet = pq.ParquetFile(filename)
        self.cols = None 
    def __getitem__(self, index):
        data = self.parquet.read_row_group(index, columns=self.cols).to_pydict()
        # Preprocessing
        #data['X'] = np.float32(data['X'][0]) 
        data['X'] = torch.Tensor(data['X'][0])
        return dict(data)
    def __len__(self):
        return self.parquet.num_row_groups

In [ ]:
def get_data_loader(datasets, batch_size, cut, random_sampler=False):
    dset = ConcatDataset([ParquetDataset(dataset) for dataset in datasets])
    idxs = np.random.permutation(len(dset))
    if random_sampler: 
        random_sampler = sampler.SubsetRandomSampler(idxs[:cut])
    else: 
        random_sampler = None 
    data_loader = DataLoader(dataset=dset, batch_size=batch_size, shuffle=False, num_workers=10, sampler=random_sampler, pin_memory=True)
    return data_loader

In [ ]:
cd drive/My\ Drive/ElPhotons

/content/drive/My Drive/ElPhotons


In [ ]:

datasets = ['ElectronData.h5.snappy.parquet','PhotonData.h5.snappy.parquet']
data_loader = get_data_loader(datasets, args.batch_size, cut = None, random_sampler = True)

In [ ]:

def jets(datei,labels,number1,number2):

    cols = None

    graphs=[]

    for i in range(number1,number2):

      ecal=datei[i].cuda()

      xhit2,yhit2=torch.nonzero(ecal,as_tuple=True)  ## Select hits in detector

      eneEcal=-torch.log(ecal[xhit2,yhit2])*10 ## Select energies of hits

      feats=torch.stack((xhit2.float(),yhit2.float(),eneEcal),dim=1) ## concatenate x,y locations and energies (3 features in total)

      cords=feats[:,[0,1]] ## cords = x,y coordinates 

      #if cords.shape[0]>=10: ## just to make sure a sample is not empty 

      edge_index = knn_graph(cords, k=6, batch=None, loop=True,num_workers=10)  ## Create knn graph adjacency matrix 

      donnees=Data(x=feats,edge_index=edge_index,label=labels[i]) ## Create graph data with feature matrix x and adjacency matrix edge_index
      
      graphs.append(donnees)

    return graphs

In [ ]:
ls

aliversion_electronphoton.ipynb
ElectronData.h5.snappy.parquet
PhotonData.h5.snappy.parquet
QuarkGluonClassification/
SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5
SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5.snappy.parquet
SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5.snappy.parquet
SinglePhotonPt50_IMGCROPS_n249k_RHv2.hdf5
test.pt


In [ ]:
import ep_model#DiffAE

In [ ]:
model=ep_model.GraphClass(3,64,128,256,128,64,32, 0.3)#DiffAE.GraphAE()
model.train()
model.to(device)#.cuda()

In [ ]:
scaler = torch.cuda.amp.GradScaler() 

In [ ]:
optimizer = torch.optim.Adam(model3.parameters(), lr=args.lr, weight_decay=0.0001)

In [ ]:

## generate list to count nodes for each graph
def nodeCounter(samples):
    inds=[]
    for k in samples:
        inds.append(k['x'].shape[0])
    return inds

In [ ]:
def ref(bsize,nodeC,i1,i2):
  maxC=np.max(np.array(nodeC))
  maxC=2000#maxC + (4 - maxC % 4) ##max num of nodes 1161%4
  refMat=np.zeros((bsize,maxC)) ## matrix of zeros
  for pi in range(i1,i2):##10
    refMat[bsize-(i2-pi),:nodeC[pi]]=1 ## fill ones 
  return refMat,maxC

In [ ]:
def assigner(nodelist):
  fin=[]
  countit=0
  for m in nodelist:
      fin.append(np.repeat(countit,m))
      countit+=1
  return np.array(fin)

In [ ]:
from optimizer import loss_function

In [ ]:
#from torch.cuda.amp import GradScaler, autocast

In [ ]:
def get_accuracy(y_true, y_prob):
    assert y_true.ndim == 1 and y_true.size() == y_prob.size()
    y_prob = y_prob > 0.5
    return (y_true == y_prob).sum().item() / y_true.size(0)

In [ ]:
import time
for epoch in range(50):
      #model.train()
  count=0
  c1,c2=0,args.batch_size
  epLoss=0
  t = time.time()
  for i, data in enumerate(data_loader):
        ecal2 = data['X'][:,0,:,:].cuda()
        labels=data['y'][0].cuda()
        rawGraph=jets(ecal2,labels,0,args.batch_size) ##Generating graphs from raw data 
        nodeCount=nodeCounter(rawGraph)
        lengs=torch.LongTensor(np.hstack(assigner(np.array(nodeCount[c1:c2])-c1))).cuda()
        
        compress=torch_geometric.data.Batch.from_data_list(rawGraph)

        gra=compress.x.to(device)
        adj=compress.edge_index.to(device)

        count+=1
        refMat,maxCount=ref(args.batch_size,nodeCount,c1,c2)
        
        optimizer.zero_grad()
        mask=torch.from_numpy(refMat).to(device) 
        #maxi=torch.from_numpy(np.array(maxCount)).to(device)

        r1= model3(gra,adj,lengs.to(device),mask,2000)

        #sparse=to_sparse_batch(r1, adj1, mask=torch.LongTensor(mask).cuda())

        loss = loss_function(r1,labels.unsqueeze_(1))/args.batch_size    

        loss.backward()

        optimizer.step()

        cur_loss = loss.item()
        
        epLoss+=float(cur_loss)

        #c1+=args.batch_size
        #c2+=args.batch_size
        if count%1250==0:
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(epLoss/count),"time=", "{:.5f}".format(time.time() - t))
            t = time.time()
        if count%10==0:
            print(get_accuracy(labels.squeeze_(1),r1.squeeze_(1)))

  torch.save({
        'epoch': epoch,
        'model_state_dict': model3.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch':epoch,
        'loss': loss,
        'epLoss':epLoss
        }, './loadBatches_b16_pool3.pth')


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0.48
0.5
0.46
0.5
0.56
0.54
0.34
0.56
0.54
0.42
0.58
0.42
0.46
0.44
0.48
0.48
0.46
0.44
0.56
0.56
0.62
0.54
0.62
0.52
0.32
0.56
0.6
0.52
0.56
0.48
0.4
0.48
0.52
0.58
0.48
0.62
0.48
0.46
0.5
0.64
0.56
0.48
0.56
0.5
0.38
0.42
0.42
0.54
0.44
0.6
0.38
0.54


KeyboardInterrupt: ignored